In [1]:
# Copyright 2021 NVIDIA Corporation. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ================================

<img src="http://developer.download.nvidia.com/compute/machine-learning/frameworks/nvidia_logo.png" style="width: 90px; float: right;">

## Building Intelligent Recommender Systems with Merlin

### Overview

Recommender Systems (RecSys) are the engine of the modern internet and the catalyst for human decisions. Building a recommendation system is challenging because it requires multiple stages (data preprocessing, offline training, item retrieval, filtering, ranking, ordering, etc.) to work together seamlessly and efficiently. The biggest challenges for new practitioners are the lack of understanding around what RecSys look like in the real world, and the gap between examples of simple models and a production-ready end-to-end recommender systems.

The figure below represents a four-stage recommender systems. This is more complex process than only training a single model and deploying it, and it is much more realistic and closer to what's happening in the real-world recommender production systems.

![fourstage](../images/fourstages.png)

In these series of notebooks, we are going to showcase how we can deploy a four-stage recommender systems using Merlin Systems library easily on [Triton Inference Server](https://github.com/triton-inference-server/server). Let's go over the concepts in the figure briefly. 
- **Retrieval:** This is the step to narrow down millions of items into thounds of candidates. We are going to train a Two-Tower item retrieval model to retrive the relevant top-K candidate items.
- **Filtering:** This step is to exclude the already interacted  or undesirable items from the candidate items set or to apply business logic rules. Although this is an important step, for this example we skip this step.
- **Scoring:** This is also known as ranking. Here the retrieved and filtered candidate items are being scored. We are going to train a ranking model to be able to use at our scoring step. 
- **Ordering:** At this stage, we can order the final set of items that we want to recommend to the user. Here, we’re able to align the output of the model with business needs, constraints, or criteria.

To learn more about the four-stage recommender systems, you can listen to Even Oldridge's [Moving Beyond Recommender Models talk](https://www.youtube.com/watch?v=5qjiY-kLwFY&list=PL65MqKWg6XcrdN4TJV0K1PdLhF_Uq-b43&index=7) at KDD'21 and read more [in this blog post](https://eugeneyan.com/writing/system-design-for-discovery/).

### Learning objectives
- Understanding four stages of recommender systems
- Training retrieval and ranking models with Merlin Models
- Setting up feature store and approximate nearest neighbours (ANN) search libraries
- Deploying trained models to Triton Inference Server with Merlin Systems

In additon to NVIDIA Merlin libraries and `Triton` library, we are using two external libraries in these series of examples:

- [Feast](https://docs.feast.dev/): an end-to-end open source feature store library for machine learning
- [Faiss](https://github.com/facebookresearch/faiss): a library for efficient similarity search and clustering of dense vectors

You can find more information about `Feast feature store` and `Faiss` libraries in the next notebook. Please follow the instructions in the README.md file to install these libraries.

### Import required libraries and functions

**Compatibility:**

These notebooks are developed and tested using our latest inference container on [NVIDIA's docker registry](https://catalog.ngc.nvidia.com/containers?filters=&orderBy=dateModifiedDESC&query=merlin).

In [3]:
%pip install tensorflow "feast<0.20" faiss-gpu

In [4]:
import os
os.environ["TF_GPU_ALLOCATOR"]="cuda_malloc_async"
import cudf
import glob
import gc

import nvtabular as nvt
from nvtabular.ops import *

from merlin.models.utils.example_utils import workflow_fit_transform

from merlin.schema.tags import Tags

import merlin.models.tf as mm
from merlin.io.dataset import Dataset
import tensorflow as tf

2022-05-04 13:26:59.541865: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-05-04 13:27:01.553019: I tensorflow/core/common_runtime/gpu/gpu_process_state.cc:214] Using CUDA malloc Async allocator for GPU: 0
2022-05-04 13:27:01.553153: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 16254 MB memory:  -> device: 0, name: Quadro GV100, pci bus id: 0000:15:00.0, compute capability: 7.0
2022-05-04 13:27:01.553841: I tensorflow/core/common_runtime/gpu/gpu_process_state.cc:214] Using CUDA malloc Async allocator for GPU: 1
2022-05-04 13:27:01.553903: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/t

In [5]:
# disable INFO and DEBUG logging everywhere
import logging
logging.disable(logging.WARNING)

In this example notebook, we will generate the synthetic train and test datasets mimicking the real [Ali-CCP: Alibaba Click and Conversion Prediction](https://tianchi.aliyun.com/dataset/dataDetail?dataId=408#1) dataset to build our recommender system models.

First, we define our input and output paths.

In [6]:
DATA_FOLDER = os.environ.get("DATA_FOLDER", "/workspace/data/")
output_path = os.path.join(DATA_FOLDER, 'processed/ranking')

Then, we use `generate_data` utility function to generate synthetic dataset. 

In [7]:
from merlin.datasets.synthetic import generate_data

NUM_ROWS = 100000
train, valid = generate_data("aliccp-raw", int(NUM_ROWS), set_sizes=(0.7, 0.3))

If you would like to use the real ALI-CCP dataset, you can use [get_aliccp()](https://github.com/NVIDIA-Merlin/models/blob/main/merlin/datasets/ecommerce/aliccp/dataset.py) function instead. This function takes the raw csv files, and generate parquet files that can be directly fed to NVTabular workflow above.

### Feature Engineering with NVTabular

We are going to process our raw categorical features by encoding them using `Categorify()` operator and tag the features with `user` or `item` tags in the schema file. To learn more about [NVTabular](https://github.com/NVIDIA-Merlin/NVTabular) and the schema object visit this example [notebook](https://github.com/NVIDIA-Merlin/models/blob/main/examples/02-Merlin-Models-and-NVTabular-integration.ipynb) in the Merlin Models repo.

In [8]:
%%time

user_id = ["user_id"] >> Categorify(dtype='int32') >> TagAsUserID()
item_id = ["item_id"] >> Categorify(dtype='int32') >> TagAsItemID()

item_features = ["item_category", "item_shop", "item_brand"] >> Categorify(dtype='int32') >> TagAsItemFeatures() 

user_features = ['user_shops', 'user_profile', 'user_group', 
       'user_gender', 'user_age', 'user_consumption_2', 'user_is_occupied',
       'user_geography', 'user_intentions', 'user_brands', 'user_categories'] \
    >> Categorify(dtype='int32') >> TagAsUserFeatures() 

targets = ["click"] >> AddMetadata(tags=[Tags.BINARY_CLASSIFICATION, "target"])

outputs = user_id+item_id+item_features+user_features+targets

CPU times: user 145 µs, sys: 46 µs, total: 191 µs
Wall time: 195 µs


Let's call `transform_aliccp` utility function to be able to perform `fit` and `transform` steps on the raw dataset applying the operators defined in the NVTabular workflow pipeline below, and also save our workflow model. After fit and transform, the processed parquet files are saved to output_path.

In [9]:
from merlin.datasets.ecommerce import transform_aliccp

transform_aliccp((train, valid), output_path, nvt_workflow=outputs, workflow_name='workflow_ranking')

/usr/lib/python3.8/site-packages/cudf/core/dataframe.py:1292: UserWarning: The deep parameter is ignored and is only included for pandas compatibility.
  warnings.warn(


### Training a Ranking Model with DLRM

NVTabular exported the schema file, `schema.pbtxt` a protobuf text file, of our processed dataset. To learn more about the schema object and schema file you can explore [02-Merlin-Models-and-NVTabular-integration.ipynb](https://github.com/NVIDIA-Merlin/models/blob/main/examples/02-Merlin-Models-and-NVTabular-integration.ipynb) notebook.

We use the `schema` object to define our model.

In [10]:
# define train and valid dataset objects
train = Dataset(os.path.join(output_path, 'train', '*.parquet'), part_size="500MB")
valid = Dataset(os.path.join(output_path, 'valid', '*.parquet'), part_size="500MB")

# define schema object
schema = train.schema

In [11]:
target_column = schema.select_by_tag(Tags.TARGET).column_names[0]
target_column

'click'

Deep Learning Recommendation Model [(DLRM)](https://arxiv.org/abs/1906.00091) architecture is a popular neural network model originally proposed by Facebook in 2019. The model was introduced as a personalization deep learning model that uses embeddings to process sparse features that represent categorical data and a multilayer perceptron (MLP) to process dense features, then interacts these features explicitly using the statistical techniques proposed in [here](https://ieeexplore.ieee.org/stamp/stamp.jsp?tp=&arnumber=5694074). To learn more about DLRM architetcture please visit `Exploring-different-models` [notebook](https://github.com/NVIDIA-Merlin/models/blob/main/examples/04-Exporting-ranking-models.ipynb) in the Merlin Models GH repo.

In [12]:
model = mm.DLRMModel(
    schema,
    embedding_dim=64,
    bottom_block=mm.MLPBlock([128, 64]),
    top_block=mm.MLPBlock([128, 64, 32]),
    prediction_tasks=mm.BinaryClassificationTask(target_column, metrics=[tf.keras.metrics.AUC()])
)

In [13]:
model.compile(optimizer='adam', run_eagerly=False)
model.fit(train, validation_data=valid, batch_size=16*1024)

2022-05-04 13:27:04.217413: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


5/5 [==============================] - 6s 224ms/step - auc: 0.4997 - loss: 0.6932 - regularization_loss: 0.0000e+00 - total_loss: 0.6932 - val_auc: 0.5000 - val_loss: 0.6931 - val_regularization_loss: 0.0000e+00 - val_total_loss: 0.6931


2022-05-04 13:27:10.905074: W tensorflow/core/grappler/optimizers/loop_optimizer.cc:907] Skipping loop optimization for Merge node with control input: cond/branch_executed/_13


We will create the feature repo in the current working directory, which is `BASE_DIR` for us.

In [14]:
# set up the base dir to for feature store
BASE_DIR = os.environ.get("BASE_DIR", "/Merlin/examples/Building-and-deploying-multi-stage-RecSys/")

Let's save our DLRM model to be able to load back at the deployment stage. 

In [15]:
model.save(os.path.join(BASE_DIR, 'dlrm'))

### Training a Retrieval Model with Two-Tower Model

Now we move to the offline retrieval stage. We are going to train a Two-Tower model for item retrieval. To learn more about the Two-tower model you can visit [05-Retrieval-Model.ipynb](https://github.com/NVIDIA-Merlin/models/blob/main/examples/05-Retrieval-Model.ipynb).

In [16]:
output_path = os.path.join(DATA_FOLDER, 'processed/retrieval')

We select only positive interaction rows where `click==1` in the dataset with `Filter()` operator.

In [17]:
user_id = ["user_id"] >> Categorify(dtype='int32') >> TagAsUserID()
item_id = ["item_id"] >> Categorify(dtype='int32') >> TagAsItemID()

item_features = ["item_category", "item_shop", "item_brand"] >> Categorify(dtype='int32') >> TagAsItemFeatures()

user_features = ['user_shops', 'user_profile', 'user_group', 
       'user_gender', 'user_age', 'user_consumption_2', 'user_is_occupied',
       'user_geography', 'user_intentions', 'user_brands', 'user_categories'] \
        >> Categorify(dtype='int32') >> TagAsUserFeatures() 

inputs = user_id + item_id + item_features + user_features + ['click'] 

outputs = inputs >> Filter(f=lambda df: df["click"] == 1)

transform_aliccp((train, valid), output_path, nvt_workflow=outputs, workflow_name='workflow_retrieval')

/usr/lib/python3.8/site-packages/cudf/core/dataframe.py:1292: UserWarning: The deep parameter is ignored and is only included for pandas compatibility.
  warnings.warn(


In [18]:
train_tt = Dataset(os.path.join(output_path, 'train', '*.parquet'))
valid_tt = Dataset(os.path.join(output_path, 'valid', '*.parquet'))

schema = train_tt.schema
schema = schema.select_by_tag([Tags.ITEM_ID, Tags.USER_ID, Tags.ITEM, Tags.USER])

In [19]:
model = mm.TwoTowerModel(
    schema,
    query_tower=mm.MLPBlock([128, 64], no_activation_last_layer=True),        
    loss="categorical_crossentropy",  
    samplers=[mm.InBatchSampler()],
    embedding_options = mm.EmbeddingOptions(infer_embedding_sizes=True),
    metrics=[mm.RecallAt(10), mm.NDCGAt(10)]
)

In [20]:
model.compile(optimizer='adam', run_eagerly=False)
model.fit(train_tt, validation_data=valid_tt, batch_size=1024*8, epochs=1)

4/5 [=======================>......] - ETA: 0s - recall_at_10: 0.0107 - ndcg_10: 0.0091 - loss: 8.9768 - regularization_loss: 0.0000e+00 - total_loss: 8.9768

2022-05-04 13:27:26.542717: W tensorflow/core/grappler/optimizers/loop_optimizer.cc:907] Skipping loop optimization for Merge node with control input: cond/branch_executed/_24


5/5 [==============================] - 6s 332ms/step - recall_at_10: 0.0125 - ndcg_10: 0.0109 - loss: 8.5304 - regularization_loss: 0.0000e+00 - total_loss: 8.5304 - val_recall_at_10: 0.0410 - val_ndcg_10: 0.0387 - val_loss: 8.7822 - val_regularization_loss: 0.0000e+00 - val_total_loss: 8.7822


In the following cells we are going to export the required user and item features files, and save the query (user) tower model and item embeddings to disk. If you want to read more about exporting retrieval models, please visit [05-Retrieval-Model.ipynb](https://github.com/NVIDIA-Merlin/models/blob/main/examples/05-Retrieval-Model.ipynb) notebook in Merlin Models library repo.

### Set up a feature store with Feast

Before we move onto the next step, we need to create a Feast feature repository.

In [21]:
!cd $BASE_DIR && feast init feature_repo


Creating a new Feast repository in /tmp/examples/feature_repo.



You should be seeing a message like <i>Creating a new Feast repository in ... </i> printed out above. Now, navigate to the `feature_repo` folder and remove the demo parquet file created by default, and `examples.py` file.

In [22]:
os.remove(os.path.join(BASE_DIR, 'feature_repo', 'example.py'))
os.remove(os.path.join(BASE_DIR, 'feature_repo/data', 'driver_stats.parquet'))

### Exporting query (user) model

In [23]:
query_tower = model.retrieval_block.query_block()
query_tower.save(os.path.join(BASE_DIR, 'query_tower'))

### Exporting user and item features

In [24]:
from merlin.models.utils.dataset import unique_rows_by_features
user_features = unique_rows_by_features(train, Tags.USER, Tags.USER_ID).compute().reset_index(drop=True)

In [25]:
user_features.head()

,user_id,user_shops,user_profile,user_group,user_gender,user_age,user_consumption_2,user_is_occupied,user_geography,user_intentions,user_brands,user_categories
0,1,1,1,1,1,1,1,1,1,1,1,1
1,2,2,1,1,1,1,1,1,1,2,2,2
2,3,3,1,1,1,1,1,1,1,3,3,3
3,4,4,1,1,1,1,1,1,1,4,4,4
4,5,5,1,1,1,1,1,1,1,5,5,5


We will artificially add `datetime` and `created` timestamp columns to our user_features dataframe. This required by Feast to track the user-item features and their creation time and to determine which version to use when we query Feast.

In [26]:
from datetime import datetime
user_features["datetime"] = datetime.now()
user_features["datetime"] = user_features["datetime"].astype("datetime64[ns]")
user_features["created"] = datetime.now()
user_features["created"] = user_features["created"].astype("datetime64[ns]")

In [27]:
user_features.head()

,user_id,user_shops,user_profile,user_group,user_gender,user_age,user_consumption_2,user_is_occupied,user_geography,user_intentions,user_brands,user_categories,datetime,created
0,1,1,1,1,1,1,1,1,1,1,1,1,2022-05-04 13:27:33.635692,2022-05-04 13:27:34.000771
1,2,2,1,1,1,1,1,1,1,2,2,2,2022-05-04 13:27:33.635692,2022-05-04 13:27:34.000771
2,3,3,1,1,1,1,1,1,1,3,3,3,2022-05-04 13:27:33.635692,2022-05-04 13:27:34.000771
3,4,4,1,1,1,1,1,1,1,4,4,4,2022-05-04 13:27:33.635692,2022-05-04 13:27:34.000771
4,5,5,1,1,1,1,1,1,1,5,5,5,2022-05-04 13:27:33.635692,2022-05-04 13:27:34.000771


In [28]:
user_features.to_parquet(os.path.join(BASE_DIR, 'feature_repo/data', 'user_features.parquet'))

In [29]:
item_features = unique_rows_by_features(train, Tags.ITEM, Tags.ITEM_ID).compute().reset_index(drop=True)

In [30]:
item_features.shape

(448, 4)

In [31]:
item_features["datetime"] = datetime.now()
item_features["datetime"] = item_features["datetime"].astype("datetime64[ns]")
item_features["created"] = datetime.now()
item_features["created"] = item_features["created"].astype("datetime64[ns]")

In [32]:
item_features.head()

,item_id,item_category,item_shop,item_brand,datetime,created
0,1,1,1,1,2022-05-04 13:27:34.097313,2022-05-04 13:27:34.101176
1,2,2,2,2,2022-05-04 13:27:34.097313,2022-05-04 13:27:34.101176
2,3,3,3,3,2022-05-04 13:27:34.097313,2022-05-04 13:27:34.101176
3,4,4,4,4,2022-05-04 13:27:34.097313,2022-05-04 13:27:34.101176
4,5,5,5,5,2022-05-04 13:27:34.097313,2022-05-04 13:27:34.101176


In [33]:
# save to disk
item_features.to_parquet(os.path.join(BASE_DIR, 'feature_repo/data', 'item_features.parquet'))

### Extract and save Item embeddings

In [34]:
item_embs = model.item_embeddings(Dataset(item_features, schema=schema), batch_size=1024)
item_embs_df = item_embs.compute(scheduler="synchronous")

In [35]:
# select only item_id together with embedding columns 
item_embeddings = item_embs_df.drop(columns=['item_category', 'item_shop', 'item_brand'])

In [36]:
item_embeddings.head()

,item_id,0,1,2,3,4,5,6,7,8,...,54,55,56,57,58,59,60,61,62,63
0,1,-0.023812,0.049087,-0.006011,-0.018138,0.076004,-0.047629,-0.016437,-0.011316,-0.010015,...,-0.023935,-0.002843,0.027594,-0.002276,-0.011821,0.011062,0.005805,-0.014591,-0.013899,0.030367
1,2,-0.026561,-0.025152,-0.057405,-0.000327,0.028478,-0.010988,-0.009221,0.001220,-0.013272,...,-0.015662,-0.002456,-0.043615,0.043939,-0.025433,-0.007775,0.029884,-0.058831,-0.011239,-0.000120
2,3,0.003156,0.002400,-0.020462,-0.004567,0.022638,-0.007897,0.011588,0.002444,-0.052003,...,0.008699,-0.032143,0.013315,-0.033496,-0.003374,0.009305,-0.002533,-0.011182,-0.022575,0.002656
3,4,0.015630,-0.026810,-0.038625,-0.009386,0.070407,-0.002853,-0.044986,-0.026715,0.029586,...,0.038602,0.005720,-0.027890,0.029849,-0.024938,-0.041847,-0.003418,-0.057658,0.015824,-0.025491
4,5,-0.029692,-0.009475,-0.058727,0.000272,0.045521,0.018626,-0.003292,-0.022988,-0.009460,...,0.023875,0.024851,-0.022909,0.022784,0.001589,0.024690,0.047870,-0.045683,-0.041048,0.019730


In [37]:
# save to disk
item_embeddings.to_parquet(os.path.join(BASE_DIR,'item_embeddings.parquet'))

### Create feature definitions 

Now we will create our user and item features definitions in the user_features.py and item_features.py files and save these files in the feature_repo.

In [38]:
# %%writefile /Merlin/examples/Building-and-deploying-multi-stage-RecSys/feature_repo/user_features.py
# from google.protobuf.duration_pb2 import Duration
# import datetime 
# from feast import Entity, Feature, FeatureView, ValueType
# from feast.infra.offline_stores.file_source import FileSource

# user_features = FileSource(
#     path="/Merlin/examples/Building-and-deploying-multi-stage-RecSys/feature_repo/data/user_features.parquet",
#     event_timestamp_column="datetime",
#     created_timestamp_column="created",
# )

# user = Entity(name="user_id", value_type=ValueType.INT32, description="user id",)

# user_features_view = FeatureView(
#     name="user_features",
#     entities=["user_id"],
#     ttl=Duration(seconds=86400 * 7),
#     features=[
#         Feature(name="user_shops", dtype=ValueType.INT32),
#         Feature(name="user_profile", dtype=ValueType.INT32),
#         Feature(name="user_group", dtype=ValueType.INT32),
#         Feature(name="user_gender", dtype=ValueType.INT32),
#         Feature(name="user_age", dtype=ValueType.INT32),
#         Feature(name="user_consumption_2", dtype=ValueType.INT32),
#         Feature(name="user_is_occupied", dtype=ValueType.INT32),
#         Feature(name="user_geography", dtype=ValueType.INT32),
#         Feature(name="user_intentions", dtype=ValueType.INT32),
#         Feature(name="user_brands", dtype=ValueType.INT32),
#         Feature(name="user_categories", dtype=ValueType.INT32),
#     ],
#     online=True,
#     input=user_features,
#     tags={},
# )

In [39]:
file = open(os.path.join(BASE_DIR, 'feature_repo/','user_features.py'), "w")
file.write(
'''
from google.protobuf.duration_pb2 import Duration
import datetime 
from feast import Entity, Feature, FeatureView, ValueType
from feast.infra.offline_stores.file_source import FileSource

user_features = FileSource(
    path="/tmp/examples/feature_repo/data/user_features.parquet",
    event_timestamp_column="datetime",
    created_timestamp_column="created",
)

user = Entity(name="user_id", value_type=ValueType.INT32, description="user id",)

user_features_view = FeatureView(
    name="user_features",
    entities=["user_id"],
    ttl=Duration(seconds=86400 * 7),
    features=[
        Feature(name="user_shops", dtype=ValueType.INT32),
        Feature(name="user_profile", dtype=ValueType.INT32),
        Feature(name="user_group", dtype=ValueType.INT32),
        Feature(name="user_gender", dtype=ValueType.INT32),
        Feature(name="user_age", dtype=ValueType.INT32),
        Feature(name="user_consumption_2", dtype=ValueType.INT32),
        Feature(name="user_is_occupied", dtype=ValueType.INT32),
        Feature(name="user_geography", dtype=ValueType.INT32),
        Feature(name="user_intentions", dtype=ValueType.INT32),
        Feature(name="user_brands", dtype=ValueType.INT32),
        Feature(name="user_categories", dtype=ValueType.INT32),
    ],
    online=True,
    input=user_features,
    tags={},
)
'''
)
file.close()

In [40]:
# %%writefile /Merlin/examples/Building-and-deploying-multi-stage-RecSys/feature_repo/item_features.py
#from google.protobuf.duration_pb2 import Duration
# import datetime 
# from feast import Entity, Feature, FeatureView, ValueType
# from feast.infra.offline_stores.file_source import FileSource

# item_features = FileSource(
#     path="/Merlin/examples/Building-and-deploying-multi-stage-RecSys/feature_repo/data/item_features.parquet",
#     event_timestamp_column="datetime",
#     created_timestamp_column="created",
# )

# item = Entity(name="item_id", value_type=ValueType.INT32, description="item id",)

# item_features_view = FeatureView(
#     name="item_features",
#     entities=["item_id"],
#     ttl=Duration(seconds=86400 * 7),
#     features=[
#         Feature(name="item_category", dtype=ValueType.INT32),
#         Feature(name="item_shop", dtype=ValueType.INT32),
#         Feature(name="item_brand", dtype=ValueType.INT32),
#     ],
#     online=True,
#     input=item_features,
#     tags={},
# )

In [41]:
with open(os.path.join(BASE_DIR, 'feature_repo/','item_features.py'), "w") as f:
    f.write(
'''
from google.protobuf.duration_pb2 import Duration
import datetime 
from feast import Entity, Feature, FeatureView, ValueType
from feast.infra.offline_stores.file_source import FileSource

item_features = FileSource(
    path="/tmp/examples/feature_repo/data/item_features.parquet",
    event_timestamp_column="datetime",
    created_timestamp_column="created",
)

item = Entity(name="item_id", value_type=ValueType.INT32, description="item id",)

item_features_view = FeatureView(
    name="item_features",
    entities=["item_id"],
    ttl=Duration(seconds=86400 * 7),
    features=[
        Feature(name="item_category", dtype=ValueType.INT32),
        Feature(name="item_shop", dtype=ValueType.INT32),
        Feature(name="item_brand", dtype=ValueType.INT32),
    ],
    online=True,
    input=item_features,
    tags={},
)
'''
    )
file.close() 

Let's checkout our Feast feature repository structure.

In [42]:
# install tree
# !apt-get update
# !apt-get install tree

In [43]:
feature_repo_path = os.path.join(BASE_DIR, 'feature_repo')
!tree $feature_repo_path

/tmp/examples/feature_repo
├── __init__.py
├── data
│   ├── item_features.parquet
│   └── user_features.parquet
├── feature_store.yaml
├── item_features.py
└── user_features.py

1 directory, 6 files


### Next Steps
We trained and exported our ranking and retrieval models and NVTabular workflows. In the next step, we will learn how to deploy our trained models into [Triton Inference Server (TIS)](https://github.com/triton-inference-server/server) with Merlin Sytems library.

For the next step, move on to the `02-Deploying-multi-stage-Recsys-with-Merlin-Systems.ipynb` notebook to deploy our saved models as an ensemble to TIS and obtain prediction results for a qiven request.

string_items = '{"ordered_ids": array([[137],\n       [311],\n       [292],\n       [332],\n       [383],\n       [233],\n       [445],\n       [297],\n       [ 93],\n       [284]], dtype=int32)}'

In [19]:
'''
Test this is our file
wahat can we do
path = {}
asdasd
how does it look
'''.format('test')

'\nTest this is our file\nwahat can we do\npath = test\nasdasd\nhow does it look\n'